In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving test_with_predictions_is_correct.csv to test_with_predictions_is_correct.csv


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# 1. Load CSV
df = pd.read_csv("test_with_predictions_is_correct.csv")

# 2. Preprocess
df = df.dropna(subset=['subreddit'])
df['is_correct'] = pd.to_numeric(df['is_correct'], errors='coerce').fillna(0).astype(int)

# 3. Ensure categorical with stable order
df['subreddit'] = pd.Categorical(df['subreddit'], categories=sorted(df['subreddit'].unique()), ordered=False)

# 4. Choose reference category
ref_cat = df['subreddit'].value_counts().idxmax()

# 5. Fit logistic regression
model = smf.logit('is_correct ~ C(subreddit, Treatment(reference="%s"))' % ref_cat, data=df).fit()
print(model.summary())

# 6. Odds ratios
odds_ratios = pd.DataFrame({
    'coef': model.params,
    'odds_ratio': model.params.apply(lambda x: round(np.exp(x), 3))
})
print("\nOdds Ratios:")
print(odds_ratios)

# 7. Stability check
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
model2 = smf.logit('is_correct ~ C(subreddit, Treatment(reference="%s"))' % ref_cat, data=df_shuffled).fit()

print("\nModel parameters approximately equal after shuffle:", np.allclose(model.params, model2.params))


Optimization terminated successfully.
         Current function value: 0.597637
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             is_correct   No. Observations:                 6256
Model:                          Logit   Df Residuals:                     6142
Method:                           MLE   Df Model:                          113
Date:                Tue, 28 Oct 2025   Pseudo R-squ.:                 0.02577
Time:                        23:58:37   Log-Likelihood:                -3738.8
converged:                       True   LL-Null:                       -3837.7
Covariance Type:            nonrobust   LLR p-value:                 1.428e-06
                                                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------